In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder


import sys
sys.path.append("..")
from utils.preprocess import *

In [3]:
exported_path = '../exported_models/'

In [8]:
input_directory = "../assets/Respeck"

data_list = []
label_list = []
total_rows = 0

for folder_name in os.listdir(input_directory):
    folder_path = os.path.join(input_directory, folder_name)
    for file in os.listdir(folder_path):
        if file.endswith(".csv") and "unprocessed" not in file:
            
            filename = os.path.join(folder_path, file)
            label, _, _ = extract_activity_and_status(filename)

            df = pd.read_csv(filename, usecols=[1,2,3])

            # Determine the number of rows to take from this file
            rows_to_take = min(700 - total_rows, len(df))
            
            # Update the df to only contain the necessary rows and update our counter
            df = df.head(rows_to_take)
            label_list.append(label)
            data_list.append(df)

# all_data = pd.concat(data_list, ignore_index=True)
# print(data_list)

In [10]:
len(data_list)

2024

In [6]:

# Normalizing sensor readings
scaler = StandardScaler()

sensor_columns = ['accel_x', 'accel_y', 'accel_z']

scaler.fit(data_list[0][sensor_columns])

for i in range(len(data_list)):
    data_list[i][sensor_columns] = scaler.transform(data_list[i][sensor_columns])


In [7]:
import numpy as np
from sklearn.model_selection import train_test_split

# Convert data and labels to numpy arrays
X = np.array([df.values for df in data_list])
X = np.stack(X)
y = LabelEncoder().fit_transform(label_list)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Convert X_train to float32
X_train = X_train.astype('float32')

In [8]:
X_train.shape

(44, 700, 3)

In [24]:
X_train[2]

array([[-3.5879586,  5.4754386, 10.459733 ],
       [-3.4432669,  5.530121 , 10.207237 ],
       [-3.445639 ,  5.565503 , 10.274356 ],
       ...,
       [-4.010174 ,  5.07015  ,  9.590381 ],
       [-3.929526 ,  5.092666 ,  9.577597 ],
       [-3.936642 ,  5.092666 ,  9.69905  ]], dtype=float32)

In [10]:
# Define input shape
input_shape = (700, 3)  # Each input sequence contains 700 time steps with 3 features.

# Number of categories
num_classes = 12


In [20]:


# # Define LSTM model
# model = models.Sequential([
#     layers.LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])),
#     layers.Dense(len(np.unique(y)), activation='relu')
# ])

# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])

# model = models.Sequential([
#     layers.Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=input_shape),
#     layers.MaxPooling1D(pool_size=2),
    
#     layers.Conv1D(filters=16, kernel_size=3, activation='relu'),
#     layers.MaxPooling1D(pool_size=2),
    
#     layers.Flatten(),
    
#     # A small Dense layer acts as the decision layer
#     layers.Dense(16, activation='relu'),
    
#     # Output layer for 12 categories. Using softmax for multi-class classification
#     layers.Dense(num_classes, activation='softmax')
# ])

model = models.Sequential([
    layers.Flatten(input_shape=input_shape),  # Flatten the input data
    layers.Dense(64, activation='relu'),  # First fully connected layer
    layers.Dense(32, activation='relu'),  # Second fully connected layer
    layers.Dense(num_classes, activation='softmax')  # Output layer for 12 categories
])

# Compile the model
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Compiling the model
# model.compile(
#     optimizer='adam',
#     loss='sparse_categorical_crossentropy',
#     metrics=['accuracy']
# )


# Train model
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2) 


Epoch 1/50
1/1 [==============================] - 0s 164ms/step - loss: 13.2598 - accuracy: 0.0286 - val_loss: 5.6800 - val_accuracy: 0.1111
Epoch 2/50
1/1 [==============================] - 0s 12ms/step - loss: 6.3247 - accuracy: 0.0857 - val_loss: 6.3671 - val_accuracy: 0.1111
Epoch 3/50
1/1 [==============================] - 0s 11ms/step - loss: 5.1922 - accuracy: 0.3143 - val_loss: 4.6727 - val_accuracy: 0.4444
Epoch 4/50
1/1 [==============================] - 0s 11ms/step - loss: 3.2547 - accuracy: 0.5714 - val_loss: 4.8393 - val_accuracy: 0.4444
Epoch 5/50
1/1 [==============================] - 0s 168ms/step - loss: 3.0573 - accuracy: 0.6000 - val_loss: 5.1918 - val_accuracy: 0.3333
Epoch 6/50
1/1 [==============================] - 0s 14ms/step - loss: 2.3949 - accuracy: 0.6571 - val_loss: 6.0723 - val_accuracy: 0.2222
Epoch 7/50
1/1 [==============================] - 0s 12ms/step - loss: 2.4126 - accuracy: 0.6286 - val_loss: 4.3366 - val_accuracy: 0.3333
Epoch 8/50
1/1 [========

In [21]:
# converter = tf.lite.TFLiteConverter.from_keras_model(model)

# tflite_model_test = converter.convert()

# with open("model.tflite", "wb") as f:
#     f.write(tflite_model_test)

converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Allow for TensorFlow ops that aren't natively supported in TFLite
# converter.target_spec.supported_ops = [
#     tf.lite.OpsSet.TFLITE_BUILTINS,
#     tf.lite.OpsSet.SELECT_TF_OPS
# ]

# # Disable the lowering of tensor list operations
# converter._experimental_lower_tensor_list_ops = False

tflite_model_test = converter.convert()

with open(exported_path + "model_dense.tflite", "wb") as f:
    f.write(tflite_model_test)


INFO:tensorflow:Assets written to: /var/folders/gg/10wpd3jj5v7dthydfjzl0gcm0000gn/T/tmpm9b7s1p8/assets


INFO:tensorflow:Assets written to: /var/folders/gg/10wpd3jj5v7dthydfjzl0gcm0000gn/T/tmpm9b7s1p8/assets
2023-10-17 15:50:12.344669: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-10-17 15:50:12.344685: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-10-17 15:50:12.344789: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/gg/10wpd3jj5v7dthydfjzl0gcm0000gn/T/tmpm9b7s1p8
2023-10-17 15:50:12.345368: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-10-17 15:50:12.345373: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/gg/10wpd3jj5v7dthydfjzl0gcm0000gn/T/tmpm9b7s1p8
2023-10-17 15:50:12.347249: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-10-17 15:50:12.373430: I tensorflow/cc/saved_model/loader.cc:215] Running initialization